# Function1

In [1]:
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
def show_work_status(singleCount, totalCount, currentCount):
    currentCount+=singleCount
    percentage=1.*currentCount/totalCount*100
    status='>'*int(percentage/2)+'-'*(50-int(percentage/2))
    if percentage % 3:
        sys.stdout.write('\rStatus:[{0}]{1:.2f}%'.format(status,percentage))
        sys.stdout.flush()
    if percentage>=100: print('\n')

In [3]:
r_part=requests.get('http://steamspy.com/api.php?request=all') 

In [4]:
dic_part = r_part.json()  
dic_part

{'570': {'appid': 570,
  'name': 'Dota 2',
  'developer': 'Valve',
  'publisher': 'Valve',
  'score_rank': '',
  'positive': 1344529,
  'negative': 260373,
  'userscore': 0,
  'owners': '100,000,000 .. 200,000,000',
  'average_forever': 35044,
  'average_2weeks': 1650,
  'median_forever': 975,
  'median_2weeks': 826,
  'price': '0',
  'initialprice': '0',
  'discount': '0',
  'ccu': 604854},
 '730': {'appid': 730,
  'name': 'Counter-Strike: Global Offensive',
  'developer': 'Valve, Hidden Path Entertainment',
  'publisher': 'Valve',
  'score_rank': '',
  'positive': 5299712,
  'negative': 713612,
  'userscore': 0,
  'owners': '50,000,000 .. 100,000,000',
  'average_forever': 29594,
  'average_2weeks': 923,
  'median_forever': 7152,
  'median_2weeks': 351,
  'price': '0',
  'initialprice': '0',
  'discount': '0',
  'ccu': 811993},
 '578080': {'appid': 578080,
  'name': 'PUBG: BATTLEGROUNDS',
  'developer': 'KRAFTON, Inc.',
  'publisher': 'KRAFTON, Inc.',
  'score_rank': '',
  'positive'

* Store the data we got as csv file

In [5]:
total_count= 0
with open('app_part.csv','w') as part_two:
    for game_id,game_details in dic_part.items():
        total_count =total_count+1
        part_two.write(json.dumps({game_id:game_details}))
        part_two.write('\n')

* Build a dict to store the data we got from http requests

In [6]:
dic_game_genre = {'appid':{},'name':{},'positive':{},'negative':{}}
current_count = 0
for game_id,game_details in dic_part.items():
    if game_details!={} and game_details is not None:
        dic_game_genre['appid'].update({game_id:game_details.get("appid",{})})
        dic_game_genre['name'].update({game_id:game_details.get('name',{})})
        dic_game_genre['positive'].update({game_id:game_details.get('positive',{})})
        dic_game_genre['negative'].update({game_id:game_details.get('negative',{})})
    show_work_status(1,total_count,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [7]:
pos = list(dic_game_genre.get('positive').values())
neg = list(dic_game_genre.get('negative').values())
appid = list(dic_game_genre.get('appid').values())
name = list(dic_game_genre.get('name').values())

In [8]:
print('Rating Based Model')
percentile_list2 = pd.DataFrame(
    {'Positive': pos,
     'Negative': neg,
     'Game_id': appid,
     'Game_name': name
    })

percentile_list1 = percentile_list2[percentile_list2['Negative']>0]
percentile_list = percentile_list1[percentile_list1['Positive']>0]
percentile_list['Total_rating'] = percentile_list['Positive'] + percentile_list['Negative']
percentile_list['Review_score'] = percentile_list['Positive']/percentile_list['Total_rating']
percentile_list['Rating'] = (percentile_list['Review_score'] - (percentile_list['Review_score'] - 0.5)*
                            2** (- np.log10(percentile_list['Total_rating'] + 1)))*100
percentile_list.sort_values("Rating", axis = 0, ascending = False, inplace = True, na_position ='first') 
percentile_list = percentile_list.reset_index(drop=True)
percentile_list

Rating Based Model


,Positive,Negative,Game_id,Game_name,Total_rating,Review_score,Rating
0,265404,3235,620,Portal 2,268639,0.987958,97.663280
1,163645,2138,1145360,Hades,165783,0.987104,97.403041
2,129738,1587,427520,Factorio,131325,0.987915,97.386900
3,71294,805,1118200,People Playground,72099,0.988835,97.197789
4,817296,17421,105600,Terraria,834717,0.979129,97.122464
...,...,...,...,...,...,...,...
992,324,542,522570,The Last Hope,866,0.374134,39.055787
993,3811,6332,681660,Bless Online,10143,0.375727,38.346080
994,2879,5802,730310,DYNASTY WARRIORS 9,8681,0.331644,34.262342
995,5489,13432,577800,NBA 2K18,18921,0.290101,30.092812


In [9]:
percentile_list.to_csv('part2.csv')

In [10]:
print('Positive Rate Based Model')
df_genre_based_results = pd.Series(list(dic_game_genre.get('positive').values()),
                         list(dic_game_genre.get('name').values())).sort_values(ascending = False)
                                       
df_genre_based_results.head(10)

Positive Rate Based Model


Counter-Strike: Global Offensive    5299712
Dota 2                              1344529
Grand Theft Auto V                  1035799
PUBG: BATTLEGROUNDS                 1020927
Tom Clancy's Rainbow Six Siege       868116
Terraria                             817296
Team Fortress 2                      771548
Garry's Mod                          732724
Rust                                 605904
The Witcher 3: Wild Hunt             550890
dtype: int64